In [5]:
%pwd

'e:\\MLs\\MLOPS Project\\SimpleMLopsProject\\research'

In [6]:
os.chdir('../')
%pwd

'e:\\MLs\\MLOPS Project\\SimpleMLopsProject'

In [38]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [39]:
from src.constants import CONFIG_FILE_PATH
from src.utils.common import read_yaml, create_directories

In [40]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(root_dir = config.root_dir,
                                                    source_url = config.source_url,
                                                    local_data_file = config.local_data_file,
                                                    unzip_dir = config.unzip_dir)
        return data_ingestion_config

In [41]:
import os
import urllib.request as request
from src import logger
import zipfile

In [42]:
## Component - data Ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: {headers}")
        else:
            logger.info(f"File {self.config.local_data_file} already exists.")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [45]:
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e

2025-07-26 21:30:31,335 - SimpleMLopsLogger - INFO - YAML file config\config.yaml read successfully.
2025-07-26 21:30:31,336 - SimpleMLopsLogger - INFO - Created directory: artifacts
2025-07-26 21:30:31,337 - SimpleMLopsLogger - INFO - Created directory: artifacts/data_ingestion
2025-07-26 21:30:31,338 - SimpleMLopsLogger - INFO - File artifacts/data_ingestion/winequality-data.zip already exists.
